# 📘 build_esp_at_docs_system.ipynb

此 Notebook 一键生成 `esp_at_docs_ci_ready/` 文件夹结构和所有配置文件，
可直接打包成 ZIP 并使用。

In [1]:
import os, zipfile, textwrap

root = 'esp_at_docs_ci_ready'
os.makedirs(f'{root}/docs/zh_CN/source/_static', exist_ok=True)
os.makedirs(f'{root}/.github/workflows', exist_ok=True)

def write(path, content):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, 'w', encoding='utf-8') as f:
        f.write(textwrap.dedent(content).strip() + '\n')

# README.md
write(f'{root}/README.md', '''
# Espressif-Style Documentation System (for Neoway)

## 🔧 Local Build
```bash
pip install -r docs/requirements.txt
cd docs
make html
```

## 🤖 CI Build (GitHub Actions)
Automatically:
- Execute `generate_at_docs.ipynb`
- Build HTML
- Deploy to GitHub Pages

## 📄 Output
`docs/zh_CN/build/html/index.html`
''')

# Makefile
write(f'{root}/docs/Makefile', '''
SPHINXBUILD   = sphinx-build
SOURCEDIR     = zh_CN/source
BUILDDIR      = zh_CN/build

.PHONY: html pdf clean

html:
	@echo "📘 Building HTML..."
	$(SPHINXBUILD) -b html $(SOURCEDIR) $(BUILDDIR)/html
	@echo "✅ HTML output in $(BUILDDIR)/html/index.html"

pdf:
	@echo "📘 Building PDF..."
	$(SPHINXBUILD) -b latexpdf $(SOURCEDIR) $(BUILDDIR)/latex
	@echo "✅ PDF output in $(BUILDDIR)/latex/"

clean:
	rm -rf $(BUILDDIR)/* || true
''')

# requirements.txt
write(f'{root}/docs/requirements.txt', '''
sphinx==8.2.3
sphinx-book-theme
sphinx-copybutton
jinja2
pyyaml
pandas
python-docx
lxml
nbconvert
reportlab
''')

# conf.py
write(f'{root}/docs/zh_CN/source/conf.py', '''
project = 'Neoway AT Command Manual'
language = 'zh_CN'
extensions = [
    'sphinx.ext.autosectionlabel',
    'sphinx_copybutton',
]
autosectionlabel_prefix_document = True
html_theme = 'sphinx_book_theme'
html_static_path = ['_static']
def setup(app):
    app.add_css_file('custom.css')
''')

# custom.css
write(f'{root}/docs/zh_CN/source/_static/custom.css', '''
.cmd-param-table th {
  background: #f2f2f2;
}
.cmd-param-table td, .cmd-param-table th {
  padding: 6px 10px;
}
.cmd-ex-table .highlight pre {
  margin: 0;
  background: #f8f9fa;
}
.cmd-ex-table td {
  vertical-align: top;
}
pre {
  background: #f9f9f9;
  border-radius: 6px;
  padding: 8px;
}
''')

# GitHub Actions workflow
write(f'{root}/.github/workflows/docs.yml', '''
name: Build Documentation

on:
  push:
    branches: [ main, optimizing-branch ]
  pull_request:

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-python@v5
        with:
          python-version: "3.11"

      - name: Install dependencies
        run: |
          pip install -r docs/requirements.txt

      - name: Run Notebook to generate RST
        run: |
          pip install jupyter nbconvert
          jupyter nbconvert --execute learning-code/csv2pdf-tool/generate_at_docs.ipynb --to notebook --output executed_notebook.ipynb

      - name: Build HTML
        run: make -C docs html

      - name: Upload Artifact
        uses: actions/upload-artifact@v4
        with:
          name: site
          path: docs/zh_CN/build/html

      - name: Deploy to GitHub Pages
        if: github.ref == 'refs/heads/main'
        uses: peaceiris/actions-gh-pages@v3
        with:
          github_token: ${{ secrets.GITHUB_TOKEN }}
          publish_dir: docs/zh_CN/build/html
          publish_branch: gh-pages
''')

# Zip it
zip_path = f'{root}.zip'
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for foldername, subfolders, filenames in os.walk(root):
        for filename in filenames:
            file_path = os.path.join(foldername, filename)
            zipf.write(file_path, os.path.relpath(file_path, root))
print(f'✅ Generated {zip_path}')
zip_path

✅ Generated esp_at_docs_ci_ready.zip


'esp_at_docs_ci_ready.zip'